Задание 1: Паттерн Command (Команда)

In [1]:
from abc import ABC, abstractmethod

# --- 1. Command Interface ---
class Command(ABC):
    """
    Интерфейс Команды объявляет метод для выполнения операции.
    """
    @abstractmethod
    def execute(self) -> None:
        pass

    @abstractmethod
    def undo(self) -> None: # Для поддержки отмены (не всегда обязательно)
        pass

# --- 2. Receiver (Получатель) ---
class Light:
    """
    Получатель содержит некоторую бизнес-логику. Почти любой объект может
    выступать Получателем.
    """
    def __init__(self, location: str = ""):
        self.location = location
        self.is_on = False

    def turn_on(self) -> None:
        self.is_on = True
        print(f"Свет {self.location} включен.")

    def turn_off(self) -> None:
        self.is_on = False
        print(f"Свет {self.location} выключен.")

class GarageDoor:
    def open(self) -> None:
        print("Дверь гаража открыта.")

    def close(self) -> None:
        print("Дверь гаража закрыта.")

# --- 3. Concrete Commands (Конкретные Команды) ---
class LightOnCommand(Command):
    """
    Конкретная команда для включения света.
    """
    def __init__(self, light: Light):
        self._light = light
        self._previous_state = light.is_on # Для undo

    def execute(self) -> None:
        self._previous_state = self._light.is_on
        self._light.turn_on()

    def undo(self) -> None:
        if self._previous_state: # Если свет был включен до команды
            self._light.turn_on()
        else:
            self._light.turn_off()
        print(f"Отмена: Свет {self._light.location} вернулся в состояние {'включен' if self._previous_state else 'выключен'}.")


class LightOffCommand(Command):
    """
    Конкретная команда для выключения света.
    """
    def __init__(self, light: Light):
        self._light = light
        self._previous_state = light.is_on

    def execute(self) -> None:
        self._previous_state = self._light.is_on
        self._light.turn_off()

    def undo(self) -> None:
        if self._previous_state:
            self._light.turn_on()
        else:
            self._light.turn_off()
        print(f"Отмена: Свет {self._light.location} вернулся в состояние {'включен' if self._previous_state else 'выключен'}.")


class GarageDoorOpenCommand(Command):
    def __init__(self, garage_door: GarageDoor):
        self._garage_door = garage_door
        # Для простоты, предположим, что до команды дверь всегда закрыта для undo
        self._was_closed = True # Упрощенное undo

    def execute(self) -> None:
        self._garage_door.open()
        self._was_closed = False # После открытия она не закрыта

    def undo(self) -> None:
        if not self._was_closed: # Если она была открыта этой командой
            self._garage_door.close()
            print("Отмена: Дверь гаража закрыта.")
        # Иначе, если она уже была открыта до команды, undo ничего не делает


class GarageDoorCloseCommand(Command):
    def __init__(self, garage_door: GarageDoor):
        self._garage_door = garage_door
        self._was_open = True # Упрощенное undo

    def execute(self) -> None:
        self._garage_door.close()
        self._was_open = False

    def undo(self) -> None:
        if not self._was_open:
            self._garage_door.open()
            print("Отмена: Дверь гаража открыта.")


class NoCommand(Command):
    """Пустая команда (Null Object Pattern)"""
    def execute(self) -> None:
        print("Нет команды для выполнения.")
    def undo(self) -> None:
        print("Нет команды для отмены.")

# --- 4. Invoker (Инициатор) ---
class RemoteControl:
    """
    Инициатор связан с одной или несколькими командами. Он отправляет запрос
    команде.
    """
    def __init__(self, num_slots: int = 7):
        self._on_commands: list[Command] = [NoCommand()] * num_slots
        self._off_commands: list[Command] = [NoCommand()] * num_slots
        self._undo_command: Command = NoCommand() # Последняя выполненная команда

    def set_command(self, slot: int, on_command: Command, off_command: Command) -> None:
        if 0 <= slot < len(self._on_commands):
            self._on_commands[slot] = on_command
            self._off_commands[slot] = off_command
        else:
            print(f"Ошибка: Слот {slot} вне диапазона.")

    def on_button_pressed(self, slot: int) -> None:
        if 0 <= slot < len(self._on_commands):
            print(f"\n--- Нажата кнопка ON для слота {slot} ---")
            self._on_commands[slot].execute()
            self._undo_command = self._on_commands[slot]
        else:
            print(f"Ошибка: Слот {slot} вне диапазона.")

    def off_button_pressed(self, slot: int) -> None:
        if 0 <= slot < len(self._off_commands):
            print(f"\n--- Нажата кнопка OFF для слота {slot} ---")
            self._off_commands[slot].execute()
            self._undo_command = self._off_commands[slot]
        else:
            print(f"Ошибка: Слот {slot} вне диапазона.")

    def undo_button_pressed(self) -> None:
        print("\n--- Нажата кнопка UNDO ---")
        self._undo_command.undo()
        self._undo_command = NoCommand() # После отмены, сбрасываем undo

    def __str__(self):
        info = "\n------ Пульт Управления ------\n"
        for i in range(len(self._on_commands)):
            info += (f"[Слот {i}] {self._on_commands[i].__class__.__name__:<25} "
                     f"{self._off_commands[i].__class__.__name__}\n")
        info += f"[UNDO] {self._undo_command.__class__.__name__}\n"
        return info

# --- Клиентский код для Задания 1 ---
if __name__ == "__main__":
    print("--- Демонстрация Паттерна Command ---")

    remote = RemoteControl(num_slots=3)

    # Создаем получателей
    living_room_light = Light("в гостиной")
    kitchen_light = Light("на кухне")
    garage_door_obj = GarageDoor()

    # Создаем команды
    living_room_light_on = LightOnCommand(living_room_light)
    living_room_light_off = LightOffCommand(living_room_light)
    kitchen_light_on = LightOnCommand(kitchen_light)
    kitchen_light_off = LightOffCommand(kitchen_light)
    garage_open = GarageDoorOpenCommand(garage_door_obj)
    garage_close = GarageDoorCloseCommand(garage_door_obj)

    # Назначаем команды слотам пульта
    remote.set_command(0, living_room_light_on, living_room_light_off)
    remote.set_command(1, kitchen_light_on, kitchen_light_off)
    remote.set_command(2, garage_open, garage_close)

    print(remote)

    # Используем пульт
    remote.on_button_pressed(0)   # Включить свет в гостиной
    remote.off_button_pressed(0)  # Выключить свет в гостиной
    remote.undo_button_pressed()  # Отменить выключение (т.е. включить)

    remote.on_button_pressed(1)   # Включить свет на кухне
    remote.undo_button_pressed()  # Отменить включение (т.е. выключить)

    remote.on_button_pressed(2)   # Открыть гараж
    remote.off_button_pressed(2)  # Закрыть гараж
    remote.undo_button_pressed()  # Отменить закрытие (т.е. открыть)
    
    # Попытка нажать кнопку для неназначенного слота или неверного
    # remote.on_button_pressed(3) # Если num_slots=3, это будет ошибка (т.к. слоты 0, 1, 2)

    print(remote) # Показать состояние команд после операций
    print("-" * 50 + "\n")

--- Демонстрация Паттерна Command ---

------ Пульт Управления ------
[Слот 0] LightOnCommand            LightOffCommand
[Слот 1] LightOnCommand            LightOffCommand
[Слот 2] GarageDoorOpenCommand     GarageDoorCloseCommand
[UNDO] NoCommand


--- Нажата кнопка ON для слота 0 ---
Свет в гостиной включен.

--- Нажата кнопка OFF для слота 0 ---
Свет в гостиной выключен.

--- Нажата кнопка UNDO ---
Свет в гостиной включен.
Отмена: Свет в гостиной вернулся в состояние включен.

--- Нажата кнопка ON для слота 1 ---
Свет на кухне включен.

--- Нажата кнопка UNDO ---
Свет на кухне выключен.
Отмена: Свет на кухне вернулся в состояние выключен.

--- Нажата кнопка ON для слота 2 ---
Дверь гаража открыта.

--- Нажата кнопка OFF для слота 2 ---
Дверь гаража закрыта.

--- Нажата кнопка UNDO ---
Дверь гаража открыта.
Отмена: Дверь гаража открыта.

------ Пульт Управления ------
[Слот 0] LightOnCommand            LightOffCommand
[Слот 1] LightOnCommand            LightOffCommand
[Слот 2] GarageD

Задание 2: Доступ к данным с Proxy и обновлением

In [2]:
import json
import os
import time
import random
import logging
from abc import ABC, abstractmethod
from typing import List, Union

# --- Настройка Логгирования ---
LOG_FILE = "data_access.log"
logging.basicConfig(filename=LOG_FILE,
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# --- Файл с данными (симуляция) ---
DATA_NUMBERS_FILE = "numbers_data.json"

def initialize_data_file(filename: str = DATA_NUMBERS_FILE, num_elements: int = 10):
    """Инициализирует или перезаписывает файл с данными."""
    data = [random.randint(1, 100) for _ in range(num_elements)]
    with open(filename, 'w') as f:
        json.dump(data, f)
    print(f"Файл данных '{filename}' инициализирован/обновлен.")

def periodically_update_data_file(filename: str = DATA_NUMBERS_FILE, interval_seconds: int = 10):
    """Функция для симуляции внешнего изменения файла данных (запускать в отдельном потоке в реальном приложении)."""
    # Для этого примера мы просто вызовем ее несколько раз вручную.
    print(f"\n[Симуляция] Файл '{filename}' будет обновлен через ~{interval_seconds} сек. (в этом примере обновляется по вызову)")
    time.sleep(0.1) # Небольшая задержка для разделения логов
    initialize_data_file(filename, random.randint(5,15))


# --- 1. Subject Interface (Интерфейс Субъекта) ---
class DataSource(ABC):
    @abstractmethod
    def get_numbers(self) -> List[Union[int, float]]:
        pass

    @abstractmethod
    def calculate_sum(self) -> Union[int, float]:
        pass

    @abstractmethod
    def calculate_max(self) -> Union[int, float]:
        pass

    @abstractmethod
    def calculate_min(self) -> Union[int, float]:
        pass

# --- 2. RealSubject (Реальный Субъект) ---
class FileDataSource(DataSource):
    def __init__(self, filename: str):
        self._filename = filename
        self._data: List[Union[int, float]] = []
        self._last_loaded_timestamp: float = 0
        if not os.path.exists(self._filename):
            initialize_data_file(self._filename) # Создаем, если нет
        self._load_data() # Первоначальная загрузка

    def _load_data(self) -> None:
        try:
            with open(self._filename, 'r') as f:
                self._data = json.load(f)
            self._last_loaded_timestamp = os.path.getmtime(self._filename)
            print(f"[FileDataSource] Данные загружены из '{self._filename}'.")
            logging.info(f"FileDataSource: Data loaded from '{self._filename}'. Contains {len(self._data)} numbers.")
        except (IOError, json.JSONDecodeError) as e:
            print(f"[FileDataSource] Ошибка загрузки данных: {e}")
            logging.error(f"FileDataSource: Error loading data from '{self._filename}': {e}")
            self._data = [] # В случае ошибки, возвращаем пустой список

    def needs_update(self) -> bool:
        """Проверяет, изменился ли файл с момента последней загрузки."""
        if not os.path.exists(self._filename):
            return False # Файла нет, нечего обновлять
        current_timestamp = os.path.getmtime(self._filename)
        return current_timestamp > self._last_loaded_timestamp

    def update_data(self) -> None:
        """Принудительно перезагружает данные из файла."""
        print("[FileDataSource] Запрос на обновление данных...")
        self._load_data()

    def get_numbers(self) -> List[Union[int, float]]:
        return self._data[:] # Возвращаем копию

    def calculate_sum(self) -> Union[int, float]:
        if not self._data: return 0
        return sum(self._data)

    def calculate_max(self) -> Union[int, float]:
        if not self._data: return float('-inf') # или None, или raise Exception
        return max(self._data)

    def calculate_min(self) -> Union[int, float]:
        if not self._data: return float('inf') # или None, или raise Exception
        return min(self._data)

# --- 3. Proxy (Заместитель) ---
class LoggingUpdatingProxy(DataSource):
    def __init__(self, real_data_source: FileDataSource):
        self._real_data_source = real_data_source
        self._log_prefix = "LoggingUpdatingProxy:"

    def _check_and_update(self) -> None:
        """Проверяет необходимость обновления и обновляет, если нужно."""
        if self._real_data_source.needs_update():
            logging.info(f"{self._log_prefix} Обнаружено изменение файла данных. Обновление...")
            print(f"[{self._log_prefix.strip(':')}] Обнаружено изменение файла данных. Обновление...")
            self._real_data_source.update_data()
        else:
            logging.info(f"{self._log_prefix} Файл данных не изменился. Обновление не требуется.")
            # print(f"[{self._log_prefix.strip(':')}] Файл данных не изменился.")


    def get_numbers(self) -> List[Union[int, float]]:
        logging.info(f"{self._log_prefix} Запрос get_numbers().")
        self._check_and_update() # Проверяем перед каждым доступом
        numbers = self._real_data_source.get_numbers()
        logging.info(f"{self._log_prefix} get_numbers() вернул {len(numbers)} элементов.")
        return numbers

    def calculate_sum(self) -> Union[int, float]:
        logging.info(f"{self._log_prefix} Запрос calculate_sum().")
        self._check_and_update()
        result = self._real_data_source.calculate_sum()
        logging.info(f"{self._log_prefix} calculate_sum() результат: {result}.")
        return result

    def calculate_max(self) -> Union[int, float]:
        logging.info(f"{self._log_prefix} Запрос calculate_max().")
        self._check_and_update()
        result = self._real_data_source.calculate_max()
        logging.info(f"{self._log_prefix} calculate_max() результат: {result}.")
        return result

    def calculate_min(self) -> Union[int, float]:
        logging.info(f"{self._log_prefix} Запрос calculate_min().")
        self._check_and_update()
        result = self._real_data_source.calculate_min()
        logging.info(f"{self._log_prefix} calculate_min() результат: {result}.")
        return result

# --- Клиентский код для Задания 2 ---
if __name__ == "__main__":
    print("\n--- Демонстрация Proxy с логгированием и обновлением ---")
    
    # Инициализация файла данных, если его нет или для чистого старта
    if os.path.exists(DATA_NUMBERS_FILE):
        os.remove(DATA_NUMBERS_FILE) # Удаляем для чистоты эксперимента
        print(f"Старый файл {DATA_NUMBERS_FILE} удален.")
    
    # Создаем реальный объект и прокси
    # Инициализация файла произойдет внутри конструктора FileDataSource, если файла нет
    real_source = FileDataSource(DATA_NUMBERS_FILE)
    proxy_source = LoggingUpdatingProxy(real_source)

    print("\nПервый доступ через прокси:")
    print(f"Числа: {proxy_source.get_numbers()}")
    print(f"Сумма: {proxy_source.calculate_sum()}")
    print(f"Максимум: {proxy_source.calculate_max()}")
    print(f"Минимум: {proxy_source.calculate_min()}")

    # Симулируем изменение файла данных "извне"
    print("\n[Симуляция] Внешнее обновление файла данных...")
    periodically_update_data_file(DATA_NUMBERS_FILE) # Обновит файл

    print("\nВторой доступ через прокси (данные должны обновиться):")
    print(f"Числа: {proxy_source.get_numbers()}")
    print(f"Сумма: {proxy_source.calculate_sum()}")

    # Еще одно изменение файла
    print("\n[Симуляция] Еще одно внешнее обновление файла данных...")
    periodically_update_data_file(DATA_NUMBERS_FILE, interval_seconds=1) 

    print("\nТретий доступ через прокси:")
    print(f"Максимум: {proxy_source.calculate_max()}")
    print(f"Минимум: {proxy_source.calculate_min()}")
    
    print(f"\nЛоги всех операций можно посмотреть в файле: {LOG_FILE}")
    print("-" * 50 + "\n")


--- Демонстрация Proxy с логгированием и обновлением ---
Файл данных 'numbers_data.json' инициализирован/обновлен.
[FileDataSource] Данные загружены из 'numbers_data.json'.

Первый доступ через прокси:
Числа: [9, 67, 82, 89, 89, 96, 77, 35, 18, 97]
Сумма: 659
Максимум: 97
Минимум: 9

[Симуляция] Внешнее обновление файла данных...

[Симуляция] Файл 'numbers_data.json' будет обновлен через ~10 сек. (в этом примере обновляется по вызову)
Файл данных 'numbers_data.json' инициализирован/обновлен.

Второй доступ через прокси (данные должны обновиться):
[LoggingUpdatingProxy] Обнаружено изменение файла данных. Обновление...
[FileDataSource] Запрос на обновление данных...
[FileDataSource] Данные загружены из 'numbers_data.json'.
Числа: [71, 14, 23, 67, 98, 37, 15, 31, 85, 45, 12]
Сумма: 498

[Симуляция] Еще одно внешнее обновление файла данных...

[Симуляция] Файл 'numbers_data.json' будет обновлен через ~1 сек. (в этом примере обновляется по вызову)
Файл данных 'numbers_data.json' инициализи

Задание 3: Приложение для работы в библиотеке

In [3]:
import json
import os
import copy
import logging
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Optional

# --- Настройка Логгирования для Библиотеки ---
LIBRARY_LOG_FILE = "library_operations.log"
library_logger = logging.getLogger("LibraryApp")
library_logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(LIBRARY_LOG_FILE, mode='w') # 'w' для перезаписи при каждом запуске
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
library_logger.addHandler(file_handler)
# library_logger.addHandler(logging.StreamHandler()) # Для вывода в консоль тоже

# === Паттерн Prototype (для Книг) ===
class Prototype(ABC):
    @abstractmethod
    def clone(self) -> Any:
        pass

class Book(Prototype):
    def __init__(self, book_id: str, title: str, author: str, year: int, copies: int = 1):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.year = year
        self.copies_available = copies # Количество доступных экземпляров
        self.total_copies = copies    # Общее количество экземпляров

    def clone(self) -> 'Book':
        # Для Book достаточно поверхностного копирования, если нет сложных вложенных изменяемых объектов
        # Если бы были, например, self.reviews = [], то нужен был бы deepcopy
        return copy.copy(self) 

    def __str__(self):
        return (f"Book ID: {self.book_id}, Title: '{self.title}', Author: {self.author}, "
                f"Year: {self.year}, Available: {self.copies_available}/{self.total_copies}")

    def to_dict(self) -> Dict[str, Any]:
        return self.__dict__

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Book':
        book = cls(data['book_id'], data['title'], data['author'], data['year'])
        book.copies_available = data.get('copies_available', data.get('copies', 1))
        book.total_copies = data.get('total_copies', data.get('copies', 1))
        return book

# === Другие Сущности ===
class Reader:
    def __init__(self, reader_id: str, name: str):
        self.reader_id = reader_id
        self.name = name
        self.borrowed_books: List[str] = [] # Список book_id

    def __str__(self):
        return f"Reader ID: {self.reader_id}, Name: {self.name}, Borrowed: {len(self.borrowed_books)} books"

    def to_dict(self) -> Dict[str, Any]:
        return self.__dict__
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Reader':
        reader = cls(data['reader_id'], data['name'])
        reader.borrowed_books = data.get('borrowed_books', [])
        return reader

class Librarian: # В данной реализации пассивен, но может иметь методы управления
    def __init__(self, librarian_id: str, name: str):
        self.librarian_id = librarian_id
        self.name = name

    def __str__(self):
        return f"Librarian ID: {self.librarian_id}, Name: {self.name}"
    
    def to_dict(self) -> Dict[str, Any]:
        return self.__dict__

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Librarian':
        return cls(data['librarian_id'], data['name'])


# === Паттерн Command ===
class LibraryCommand(ABC):
    def __init__(self, library_manager: 'LibraryManagerFacade'): # Forward declaration
        self.library_manager = library_manager

    @abstractmethod
    def execute(self) -> Any:
        pass
    
    # undo можно добавить по аналогии с Заданием 1

class AddBookCommand(LibraryCommand):
    def __init__(self, library_manager: 'LibraryManagerFacade', book: Book):
        super().__init__(library_manager)
        self.book = book

    def execute(self) -> bool:
        return self.library_manager.add_book_internal(self.book)

class BorrowBookCommand(LibraryCommand):
    def __init__(self, library_manager: 'LibraryManagerFacade', book_id: str, reader_id: str):
        super().__init__(library_manager)
        self.book_id = book_id
        self.reader_id = reader_id
        
    def execute(self) -> bool:
        return self.library_manager.borrow_book_internal(self.book_id, self.reader_id)

class ReturnBookCommand(LibraryCommand):
    def __init__(self, library_manager: 'LibraryManagerFacade', book_id: str, reader_id: str):
        super().__init__(library_manager)
        self.book_id = book_id
        self.reader_id = reader_id
        
    def execute(self) -> bool:
        return self.library_manager.return_book_internal(self.book_id, self.reader_id)


# === Паттерн Strategy (для Поиска) ===
class SearchStrategy(ABC):
    @abstractmethod
    def search(self, books: List[Book], query: str) -> List[Book]:
        pass

class SearchByTitleStrategy(SearchStrategy):
    def search(self, books: List[Book], query: str) -> List[Book]:
        query_lower = query.lower()
        return [book for book in books if query_lower in book.title.lower()]

class SearchByAuthorStrategy(SearchStrategy):
    def search(self, books: List[Book], query: str) -> List[Book]:
        query_lower = query.lower()
        return [book for book in books if query_lower in book.author.lower()]

class SearchByBookIDStrategy(SearchStrategy):
    def search(self, books: List[Book], query: str) -> List[Book]:
        return [book for book in books if query == book.book_id]

# === Паттерн Singleton (для LibraryManager) и Facade ===
# Реализация Singleton через метакласс
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class LibraryManager(metaclass=SingletonMeta): # Реальный менеджер, скрытый за фасадом
    def __init__(self):
        self.books: Dict[str, Book] = {} # book_id -> Book object
        self.readers: Dict[str, Reader] = {} # reader_id -> Reader object
        self.librarians: Dict[str, Librarian] = {} # librarian_id -> Librarian object
        self.data_file_books = "library_books.json"
        self.data_file_readers = "library_readers.json"
        library_logger.info("LibraryManager initialized.")
        # self.load_all_data() # Можно загружать при инициализации

    # --- Внутренние методы (для Command и Facade) ---
    def _add_book(self, book: Book) -> bool:
        if book.book_id in self.books:
            # Если книга существует, увеличиваем количество копий
            self.books[book.book_id].total_copies += book.total_copies
            self.books[book.book_id].copies_available += book.copies_available
            library_logger.info(f"Copies for book ID '{book.book_id}' updated. Total: {self.books[book.book_id].total_copies}")
            return True
        self.books[book.book_id] = book
        library_logger.info(f"Book added: {book.title} (ID: {book.book_id})")
        return True

    def _remove_book(self, book_id: str) -> bool:
        if book_id in self.books:
            # Проверка, не на руках ли книга
            for reader in self.readers.values():
                if book_id in reader.borrowed_books:
                    library_logger.warning(f"Attempt to remove borrowed book ID '{book_id}'. Denied.")
                    print(f"Ошибка: Книга '{self.books[book_id].title}' находится у читателя {reader.name}.")
                    return False
            del self.books[book_id]
            library_logger.info(f"Book removed: ID {book_id}")
            return True
        library_logger.warning(f"Attempt to remove non-existent book ID '{book_id}'.")
        return False

    def _add_reader(self, reader: Reader) -> bool:
        if reader.reader_id in self.readers:
            library_logger.warning(f"Reader ID '{reader.reader_id}' already exists.")
            return False
        self.readers[reader.reader_id] = reader
        library_logger.info(f"Reader added: {reader.name} (ID: {reader.reader_id})")
        return True

    def _borrow_book(self, book_id: str, reader_id: str) -> bool:
        book = self.books.get(book_id)
        reader = self.readers.get(reader_id)
        if not book:
            library_logger.warning(f"Borrow attempt: Book ID '{book_id}' not found.")
            return False
        if not reader:
            library_logger.warning(f"Borrow attempt: Reader ID '{reader_id}' not found.")
            return False
        if book.copies_available > 0:
            book.copies_available -= 1
            reader.borrowed_books.append(book_id)
            library_logger.info(f"Book '{book.title}' borrowed by reader '{reader.name}'.")
            return True
        library_logger.warning(f"Borrow attempt: No copies of '{book.title}' available.")
        return False
        
    def _return_book(self, book_id: str, reader_id: str) -> bool:
        book = self.books.get(book_id)
        reader = self.readers.get(reader_id)
        if not book or not reader or book_id not in reader.borrowed_books:
            library_logger.warning(f"Return attempt failed: Book ID '{book_id}' or Reader ID '{reader_id}' invalid or book not borrowed by reader.")
            return False
        book.copies_available += 1
        reader.borrowed_books.remove(book_id)
        library_logger.info(f"Book '{book.title}' returned by reader '{reader.name}'.")
        return True

    # --- Сохранение/Загрузка ---
    def _save_data(self, data_dict: Dict, filename: str):
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data_dict, f, ensure_ascii=False, indent=4)
            library_logger.info(f"Data saved to {filename}")
        except IOError as e:
            library_logger.error(f"Error saving to {filename}: {e}")

    def _load_data(self, filename: str, from_dict_method) -> Dict:
        if not os.path.exists(filename):
            library_logger.warning(f"Load attempt: File {filename} not found.")
            return {}
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                raw_data = json.load(f)
                # Преобразуем словари обратно в объекты
                loaded_objects = {key: from_dict_method(val) for key, val in raw_data.items()}
            library_logger.info(f"Data loaded from {filename}")
            return loaded_objects
        except (IOError, json.JSONDecodeError) as e:
            library_logger.error(f"Error loading from {filename}: {e}")
            return {}

    def save_books(self):
        books_to_save = {bid: book.to_dict() for bid, book in self.books.items()}
        self._save_data(books_to_save, self.data_file_books)

    def load_books(self):
        self.books = self._load_data(self.data_file_books, Book.from_dict)

    def save_readers(self):
        readers_to_save = {rid: reader.to_dict() for rid, reader in self.readers.items()}
        self._save_data(readers_to_save, self.data_file_readers)

    def load_readers(self):
        self.readers = self._load_data(self.data_file_readers, Reader.from_dict)
        
    def load_all_data(self):
        self.load_books()
        self.load_readers()
        library_logger.info("All library data loaded.")

    def save_all_data(self):
        self.save_books()
        self.save_readers()
        library_logger.info("All library data saved.")

# === Facade (для удобного интерфейса) + Логгирующий Декоратор/Proxy ===
# Можно использовать декоратор для логирования вызовов методов Фасада

def log_action_decorator(method):
    def wrapper(self: 'LibraryManagerFacade', *args, **kwargs):
        action_name = method.__name__
        # Формируем сообщение о параметрах, избегая вывода самого self
        arg_strings = [repr(arg) for arg in args]
        kwarg_strings = [f"{k}={repr(v)}" for k, v in kwargs.items()]
        params_string = ", ".join(arg_strings + kwarg_strings)
        
        library_logger.info(f"Facade action invoked: {action_name}({params_string})")
        try:
            result = method(self, *args, **kwargs)
            library_logger.info(f"Facade action {action_name} completed. Result: {repr(result)}")
            return result
        except Exception as e:
            library_logger.error(f"Facade action {action_name} failed: {e}")
            raise # Перевыбрасываем исключение
    return wrapper


class LibraryManagerFacade:
    def __init__(self):
        self._manager = LibraryManager() # Получаем Singleton экземпляр
        self._command_history: List[LibraryCommand] = []
        self._search_strategy: Optional[SearchStrategy] = None # По умолчанию нет стратегии

    def _execute_command(self, command: LibraryCommand) -> Any:
        result = command.execute()
        if result: # Или если команда всегда что-то возвращает
             self._command_history.append(command) # Для возможного undo/redo или просто истории
        return result

    @log_action_decorator
    def add_book(self, book: Book) -> bool:
        command = AddBookCommand(self, book) # Передаем себя (фасад) в команду
        return self._execute_command(command)
    
    # Внутренние методы, которые вызываются командами, не должны быть в фасаде напрямую,
    # если мы хотим строго следовать Command паттерну, где команды работают с Receiver (LibraryManager)
    # Но для упрощения взаимодействия, команды могут вызывать методы фасада, которые делегируют Receiver'у
    # Либо команды напрямую работают с _manager. Для этого им нужно передавать _manager.
    # Сделаем так, чтобы команды работали с фасадом, а фасад с менеджером.
    
    def add_book_internal(self, book: Book) -> bool: # Вызывается командой AddBookCommand
        return self._manager._add_book(book)
    
    @log_action_decorator
    def remove_book(self, book_id: str) -> bool:
        return self._manager._remove_book(book_id) # Пока без команды для простоты

    @log_action_decorator
    def add_reader(self, reader: Reader) -> bool:
        return self._manager._add_reader(reader)

    @log_action_decorator
    def borrow_book(self, book_id: str, reader_id: str) -> bool:
        command = BorrowBookCommand(self, book_id, reader_id)
        return self._execute_command(command)

    def borrow_book_internal(self, book_id: str, reader_id: str) -> bool: # Вызывается командой
        return self._manager._borrow_book(book_id, reader_id)
        
    @log_action_decorator
    def return_book(self, book_id: str, reader_id: str) -> bool:
        command = ReturnBookCommand(self, book_id, reader_id)
        return self._execute_command(command)

    def return_book_internal(self, book_id: str, reader_id: str) -> bool: # Вызывается командой
        return self._manager._return_book(book_id, reader_id)

    @log_action_decorator
    def set_search_strategy(self, strategy: SearchStrategy):
        self._search_strategy = strategy
        library_logger.info(f"Search strategy set to: {strategy.__class__.__name__}")

    @log_action_decorator
    def search_books(self, query: str, output_to_file: Optional[str] = None) -> List[Book]:
        if not self._search_strategy:
            library_logger.warning("Search attempt without a strategy set.")
            print("Ошибка: Стратегия поиска не установлена.")
            return []
        
        all_books = list(self._manager.books.values())
        results = self._search_strategy.search(all_books, query)
        
        print(f"\nРезультаты поиска для '{query}' (стратегия: {self._search_strategy.__class__.__name__}):")
        if not results:
            print("Ничего не найдено.")
        else:
            for book in results:
                print(f"  - {book}")
        
        if output_to_file:
            try:
                with open(output_to_file, 'w', encoding='utf-8') as f:
                    f.write(f"Search query: {query}\nStrategy: {self._search_strategy.__class__.__name__}\n\n")
                    if not results:
                        f.write("No results found.\n")
                    else:
                        for book in results:
                            f.write(str(book) + "\n")
                library_logger.info(f"Search results for '{query}' saved to {output_to_file}")
                print(f"Результаты поиска также сохранены в файл: {output_to_file}")
            except IOError as e:
                library_logger.error(f"Error saving search results to {output_to_file}: {e}")
                print(f"Ошибка сохранения результатов поиска в файл: {e}")
        return results

    @log_action_decorator
    def save_all_data(self):
        self._manager.save_all_data()
    
    @log_action_decorator
    def load_all_data(self):
        self._manager.load_all_data()

    @log_action_decorator
    def display_all_books(self):
        print("\n--- Все книги в библиотеке ---")
        if not self._manager.books:
            print("В библиотеке нет книг.")
            return
        for book in self._manager.books.values():
            print(book)
            
    @log_action_decorator
    def display_all_readers(self):
        print("\n--- Все читатели ---")
        if not self._manager.readers:
            print("Нет зарегистрированных читателей.")
            return
        for reader in self._manager.readers.values():
            print(reader)


# --- Клиентский код для Задания 3 ---
if __name__ == "__main__":
    print("\n--- Демонстрация Приложения Библиотеки ---")
    
    # Очистка лог-файла и файлов данных для чистоты эксперимента при каждом запуске
    if os.path.exists(LIBRARY_LOG_FILE): os.remove(LIBRARY_LOG_FILE)
    if os.path.exists("library_books.json"): os.remove("library_books.json")
    if os.path.exists("library_readers.json"): os.remove("library_readers.json")

    library_facade = LibraryManagerFacade() # Создаем/получаем фасад (и Singleton менеджера)

    # Загрузка данных (файлов еще нет, ничего не загрузится)
    library_facade.load_all_data() 

    # Добавляем книги
    book1 = Book("B001", "Война и мир", "Лев Толстой", 1869, 3)
    book2 = Book("B002", "Преступление и наказание", "Федор Достоевский", 1866, 2)
    book3 = Book("B003", "Мастер и Маргарита", "Михаил Булгаков", 1967, 5)
    book4_prototype = Book("B004P", "Прототипная Книга", "Автор Прототипов", 2024, 1)


    library_facade.add_book(book1)
    library_facade.add_book(book2)
    library_facade.add_book(book3)
    
    # Используем Prototype для создания еще одной копии книги
    book4_clone1 = book4_prototype.clone()
    book4_clone1.book_id = "B004_C1" 
    book4_clone1.copies_available = 2 # Изменяем клон
    book4_clone1.total_copies = 2
    library_facade.add_book(book4_clone1)
    
    # Добавляем еще один экземпляр книги, ID которой уже есть (увеличит кол-во копий)
    library_facade.add_book(Book("B001", "Война и мир", "Лев Толстой", 1869, 2))


    # Добавляем читателей
    reader1 = Reader("R001", "Иван Иванов")
    reader2 = Reader("R002", "Мария Петрова")
    library_facade.add_reader(reader1)
    library_facade.add_reader(reader2)

    library_facade.display_all_books()
    library_facade.display_all_readers()

    # Выдача книг
    print("\n--- Выдача книг ---")
    library_facade.borrow_book("B001", "R001") # Иван берет "Война и мир"
    library_facade.borrow_book("B003", "R001") # Иван берет "Мастер и Маргарита"
    library_facade.borrow_book("B002", "R002") # Мария берет "Преступление и наказание"
    library_facade.borrow_book("B001", "R002") # Мария пытается взять "Война и мир" (еще есть копии)
    
    library_facade.display_all_books()
    library_facade.display_all_readers()

    # Возврат книг
    print("\n--- Возврат книг ---")
    library_facade.return_book("B003", "R001") # Иван возвращает "Мастер и Маргарита"

    library_facade.display_all_books()
    library_facade.display_all_readers()

    # Поиск книг
    print("\n--- Поиск книг ---")
    library_facade.set_search_strategy(SearchByTitleStrategy())
    library_facade.search_books("мир", output_to_file="search_title_results.txt")

    library_facade.set_search_strategy(SearchByAuthorStrategy())
    library_facade.search_books("Толстой")
    
    library_facade.set_search_strategy(SearchByBookIDStrategy())
    library_facade.search_books("B002", output_to_file="search_id_results.txt")
    
    # Сохранение всех данных
    library_facade.save_all_data()
    
    # Симуляция закрытия и нового открытия приложения
    print("\n--- Симуляция перезапуска: Создаем новый фасад и загружаем данные ---")
    # Старый объект facade все еще существует, но LibraryManager - Singleton
    new_library_facade = LibraryManagerFacade() 
    print(f"ID старого менеджера: {id(library_facade._manager)}")
    print(f"ID нового менеджера (должен быть тот же из-за Singleton): {id(new_library_facade._manager)}")
    
    new_library_facade.load_all_data() # Загружаем сохраненные данные
    new_library_facade.display_all_books()
    new_library_facade.display_all_readers()
    
    # Удаление книги
    print("\n--- Удаление книги ---")
    library_facade.remove_book("B004_C1") # Удаляем клон
    library_facade.remove_book("B001")    # Пытаемся удалить книгу, которая на руках у читателей
    new_library_facade.display_all_books()

    print(f"\nЛоги всех операций библиотеки можно посмотреть в файле: {LIBRARY_LOG_FILE}")
    print("-" * 50 + "\n")


--- Демонстрация Приложения Библиотеки ---

--- Все книги в библиотеке ---
Book ID: B001, Title: 'Война и мир', Author: Лев Толстой, Year: 1869, Available: 5/5
Book ID: B002, Title: 'Преступление и наказание', Author: Федор Достоевский, Year: 1866, Available: 2/2
Book ID: B003, Title: 'Мастер и Маргарита', Author: Михаил Булгаков, Year: 1967, Available: 5/5
Book ID: B004_C1, Title: 'Прототипная Книга', Author: Автор Прототипов, Year: 2024, Available: 2/2

--- Все читатели ---
Reader ID: R001, Name: Иван Иванов, Borrowed: 0 books
Reader ID: R002, Name: Мария Петрова, Borrowed: 0 books

--- Выдача книг ---

--- Все книги в библиотеке ---
Book ID: B001, Title: 'Война и мир', Author: Лев Толстой, Year: 1869, Available: 3/5
Book ID: B002, Title: 'Преступление и наказание', Author: Федор Достоевский, Year: 1866, Available: 1/2
Book ID: B003, Title: 'Мастер и Маргарита', Author: Михаил Булгаков, Year: 1967, Available: 4/5
Book ID: B004_C1, Title: 'Прототипная Книга', Author: Автор Прототипов